In [1]:
# Imports at the top
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
%matplotlib inline

# Scraping and Importing IMDB Data

In [2]:
# function to get Top 250 movies on IMDB 
# Scraping 
def get_top_250():
    response = requests.get('http://www.imdb.com/chart/top')
    html = response.text
    entries = re.findall("<a href.*?/title/(.*?)/", html)
    return list(set(entries))

In [3]:
# set entries equal to top 250 from scraped files
entries = get_top_250()

In [4]:
# get details and entries for IMDB movies, API

def get_entry(entry):
    res = requests.get('http://www.omdbapi.com/?i='+entry)
    if res.status_code != 200:
        print entry, res.status_code
    else:
        print '.',
    try:
        j = json.loads(res.text)
    except ValueError:
        j = None
    return j

In [5]:
entries_dict_list = [get_entry(e) for e in entries]

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


In [6]:
# convert movie entires to dataframe
df = pd.DataFrame(entries_dict_list)

In [7]:
df1 = pd.read_csv('imdb_grossrev.csv', index_col=0)

# Data Cleaning & Munging

In [8]:
df.head(4)

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,Released,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes
0,"Miles Teller, J.K. Simmons, Paul Reiser, Melis...",Won 3 Oscars. Another 87 wins & 131 nominations.,USA,Damien Chazelle,"Drama, Music",English,88,A promising young drummer enrolls at a cut-thr...,https://images-na.ssl-images-amazon.com/images...,R,15 Oct 2014,True,107 min,Whiplash,movie,Damien Chazelle,2014,tt2582802,8.5,"413,720"
1,"Toshirô Mifune, Takashi Shimura, Keiko Tsushim...",Nominated for 2 Oscars. Another 5 wins & 6 nom...,Japan,Akira Kurosawa,"Action, Adventure, Drama",Japanese,98,A poor village under attack by bandits recruit...,https://images-na.ssl-images-amazon.com/images...,UNRATED,19 Nov 1956,True,207 min,Seven Samurai,movie,"Akira Kurosawa (screenplay), Shinobu Hashimoto...",1954,tt0047478,8.7,"232,249"
2,"Harrison Ford, Karen Allen, Paul Freeman, Rona...",Won 4 Oscars. Another 30 wins & 23 nominations.,USA,Steven Spielberg,"Action, Adventure","English, German, Hebrew, Spanish, Arabic, Nepali",85,Archaeologist and adventurer Indiana Jones is ...,https://images-na.ssl-images-amazon.com/images...,PG,12 Jun 1981,True,115 min,Raiders of the Lost Ark,movie,"Lawrence Kasdan (screenplay), George Lucas (st...",1981,tt0082971,8.5,"671,034"
3,"William Holden, Alec Guinness, Jack Hawkins, S...",Won 7 Oscars. Another 23 wins & 7 nominations.,"UK, USA",David Lean,"Adventure, Drama, War","English, Japanese, Thai",N/A,After settling his differences with a Japanese...,https://images-na.ssl-images-amazon.com/images...,PG,14 Dec 1957,True,161 min,The Bridge on the River Kwai,movie,"Pierre Boulle (novel), Carl Foreman (screenpla...",1957,tt0050212,8.2,"151,604"


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 20 columns):
Actors        250 non-null object
Awards        250 non-null object
Country       250 non-null object
Director      250 non-null object
Genre         250 non-null object
Language      250 non-null object
Metascore     250 non-null object
Plot          250 non-null object
Poster        250 non-null object
Rated         250 non-null object
Released      250 non-null object
Response      250 non-null object
Runtime       250 non-null object
Title         250 non-null object
Type          250 non-null object
Writer        250 non-null object
Year          250 non-null object
imdbID        250 non-null object
imdbRating    250 non-null object
imdbVotes     250 non-null object
dtypes: object(20)
memory usage: 39.1+ KB


In [10]:
# Replace N/A with NaN
df = df.replace('N/A', np.nan)

In [11]:
# convert released column to datetime type
df['Released'] = pd.to_datetime(df['Released'])

In [12]:
# strip 'min' from runtime column and convert to integer
def intminutes(x):
    y = x.replace('min','').strip()
    return int(y)

df['Runtime'] = df['Runtime'].apply(intminutes)

In [13]:
# convert year column to integer types
df['Year'] = df['Year'].astype(int)

In [14]:
# convert year column to integer types
df['Metascore'] = df['Metascore'].astype(float)

In [15]:
# strip commas from imdbVotes and convert to integer
def intvotes(x):
    y = x.replace(',', '').strip()
    return int(y)
df.imdbVotes = df.imdbVotes.apply(intvotes)

In [16]:
# combine IMDB dataframe with IMDB Gross Revenue dataframe
df = pd.merge(df, df1)

## Replace NaN with Values

In [17]:
# return all rows with NaN
df_nan = df[pd.isnull(df).any(axis = 1)]
df_nan

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,...,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,Gross
3,"William Holden, Alec Guinness, Jack Hawkins, S...",Won 7 Oscars. Another 23 wins & 7 nominations.,"UK, USA",David Lean,"Adventure, Drama, War","English, Japanese, Thai",NaN,After settling his differences with a Japanese...,https://images-na.ssl-images-amazon.com/images...,PG,...,True,161,The Bridge on the River Kwai,movie,"Pierre Boulle (novel), Carl Foreman (screenpla...",1957,tt0050212,8.2,151604,27200000.0
7,"Yves Montand, Charles Vanel, Folco Lulli, Pete...",Won 1 BAFTA Film Award. Another 5 wins.,"France, Italy",Henri-Georges Clouzot,"Adventure, Drama, Thriller","French, English, Spanish, German, Italian, Rus...",NaN,"In a decrepit South American village, men are ...",https://images-na.ssl-images-amazon.com/images...,NOT RATED,...,True,131,The Wages of Fear,movie,"Georges Arnaud (novel), Henri-Georges Clouzot ...",1953,tt0046268,8.2,35833,NaN
8,"Toshirô Mifune, Tatsuya Nakadai, Yôko Tsukasa,...",Nominated for 1 Oscar. Another 4 wins & 1 nomi...,Japan,Akira Kurosawa,"Action, Drama, Thriller",Japanese,NaN,A crafty ronin comes to a town divided by two ...,https://images-na.ssl-images-amazon.com/images...,UNRATED,...,True,110,Yojimbo,movie,"Akira Kurosawa (story), Akira Kurosawa (screen...",1961,tt0055630,8.3,75317,NaN
10,"Bette Davis, Anne Baxter, George Sanders, Cele...",Won 6 Oscars. Another 16 wins & 17 nominations.,USA,Joseph L. Mankiewicz,Drama,"English, French",NaN,An ingenue insinuates herself into the company...,https://images-na.ssl-images-amazon.com/images...,APPROVED,...,True,138,All About Eve,movie,Joseph L. Mankiewicz (written for the screen by),1950,tt0042192,8.3,84353,10177.0
14,"Steve McQueen, James Garner, Richard Attenboro...",Nominated for 1 Oscar. Another 2 wins & 7 nomi...,USA,John Sturges,"Adventure, Drama, History","English, German, French, Russian, Spanish",NaN,Allied P.O.W.s plan for several hundred of the...,https://images-na.ssl-images-amazon.com/images...,APPROVED,...,True,172,The Great Escape,movie,"Paul Brickhill (book), James Clavell (screenpl...",1963,tt0057115,8.3,167926,NaN
16,"Jack Nicholson, Faye Dunaway, John Huston, Per...",Won 1 Oscar. Another 20 wins & 23 nominations.,USA,Roman Polanski,"Drama, Mystery, Thriller","English, Cantonese, Spanish",86.0,A private detective hired to expose an adulter...,https://images-na.ssl-images-amazon.com/images...,R,...,True,130,Chinatown,movie,Robert Towne,1974,tt0071315,8.2,220193,NaN
21,"Anthony Quinn, Irene Papas, Michael Ansara, Jo...",NaN,"Lebanon, Libya, Kuwait, Morocco, UK",Moustapha Akkad,"Adventure, Biography, Drama","English, Arabic",NaN,"In the 7th century, Mohammed is visited by Ang...",https://images-na.ssl-images-amazon.com/images...,PG,...,True,177,The Message,movie,"H.A.L. Craig, Tewfik El-Hakim, A.B. Jawdat El-...",1976,tt0074896,8.4,31785,NaN
24,"Virginia Cherrill, Florence Lee, Harry Myers, ...",3 wins.,USA,Charles Chaplin,"Comedy, Drama, Romance",English,NaN,"With the aid of a wealthy erratic tippler, a d...",https://images-na.ssl-images-amazon.com/images...,PASSED,...,True,87,City Lights,movie,Charles Chaplin,1931,tt0021749,8.6,109921,NaN
27,"Cary Grant, Eva Marie Saint, James Mason, Jess...",Nominated for 3 Oscars. Another 8 wins & 4 nom...,USA,Alfred Hitchcock,"Action, Adventure, Crime",English,NaN,A hapless New York advertising executive is mi...,http://ia.media-imdb.com/images/M/MV5BMjQwMTQ0...,APPROVED,...,True,136,North by Northwest,movie,Ernest Lehman,1959,tt0053125,8.4,221404,NaN
31,"Donnie Yen, Simon Yam, Lynn Hung, Hiroyuki Ike...",12 wins & 15 nominations.,"Hong Kong, China",Wilson Yip,"Action, Biography, Drama","Cantonese, Mandarin, Japanese",59.0,"During the Japanese invasion of 1937, when a w...",https://images-na.ssl-images-amazon.com/images...,R,...,True,106,Ip Man,movie,"Edmond Wong (screenplay), Tai-Li Chan",2008,tt1220719,8.1,171712,NaN


In [18]:
# Print number of NaN in all columns

nan_list = []

for col in df.columns:
    x = sum(pd.isnull(df[col]))
    if x != 0:
        nan_list.append(col)
        print col, x

Awards 4
Language 1
Metascore 81
Poster 2
Rated 1
Released 1
Gross 66


In [19]:
# Find row where Language is NaN
df[pd.isnull(df['Language'])]

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,...,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,Gross
194,"George O'Brien, Janet Gaynor, Margaret Livings...",Won 3 Oscars. Another 2 wins & 1 nomination.,USA,F.W. Murnau,"Drama, Romance",NaN,NaN,A married farmer falls under the spell of a sl...,NaN,NOT RATED,...,True,94,Sunrise,movie,"Carl Mayer (scenario), Hermann Sudermann (from...",1927,tt0018455,8.3,29635,NaN


In [20]:
# Change language of movie "Sunrise" to "English"
df.loc[df['Title'] == 'Sunrise' , 'Language'] = 'English'

In [21]:
# Find movie with NaN in Rating

df[pd.isnull(df['Rated'])]

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,...,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,Gross
137,"Manoj Bajpayee, Richa Chadha, Nawazuddin Siddi...",10 wins & 32 nominations.,India,Anurag Kashyap,"Action, Crime, Drama",Hindi,89.0,A clash between Sultan and Shahid Khan leads t...,https://images-na.ssl-images-amazon.com/images...,NaN,...,True,320,Gangs of Wasseypur,movie,"Akhilesh Jaiswal, Anurag Kashyap, Sachin K. La...",2012,tt1954470,8.3,48140,NaN


In [22]:
# List of all unique ratings
df['Rated'].unique()

array([u'R', u'UNRATED', u'PG', u'PG-13', u'NOT RATED', u'G', u'APPROVED',
       u'PASSED', nan, u'M'], dtype=object)

In [23]:
# Change rating of 'Gangs of Wasseypur' from NaN to 'UNRATED'

df.loc[df['Title'] == 'Gangs of Wasseypur' , 'Rated'] = 'UNRATED'

In [24]:
# Find movie with NaN in "Released"

df[pd.isnull(df['Released'])]

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,...,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,Gross
73,"Charles Chaplin, Mack Swain, Tom Murray, Henry...",Nominated for 2 Oscars. Another 3 wins & 1 nom...,USA,Charles Chaplin,"Adventure, Comedy, Drama",English,NaN,A prospector goes to the Klondike in search of...,https://images-na.ssl-images-amazon.com/images...,NOT RATED,...,True,95,The Gold Rush,movie,Charles Chaplin,1925,tt0015864,8.2,69196,NaN


In [25]:
# Change released date of  of 'The Gold Rush' from NaN to '1925-06-26'

df.loc[df['Title'] == 'The Gold Rush' , 'Released'] = pd.to_datetime('1925-06-26')

In [26]:
# Fill in 0 for NaN Gross Revenue values

df['Gross'].fillna(0, inplace=True)

In [27]:
# Drop "Awards" column - colinearity of awards and rating
df.drop('Awards',axis=1, inplace=True)

In [28]:
# Drop "Poster" column - image type
df.drop('Poster',axis=1, inplace=True)

In [29]:
# Drop "Metascore" column - Colinearity/NaN
df.drop('Metascore',axis=1, inplace=True)

In [30]:
# Drop "imdbID" column - just IDs
df.drop('imdbID',axis=1, inplace=True)

In [31]:
# Drop "Writer" column - Too many different writers
df.drop('Writer',axis=1, inplace=True)

In [32]:
# Drop "Type" column - All type = movies
df.drop('Type',axis=1, inplace=True)

In [33]:
# Drop "Response" column - All Response = True
df.drop('Response',axis=1, inplace=True)

In [34]:
df.head()

,Actors,Country,Director,Genre,Language,Plot,Rated,Released,Runtime,Title,Year,imdbRating,imdbVotes,Gross
0,"Miles Teller, J.K. Simmons, Paul Reiser, Melis...",USA,Damien Chazelle,"Drama, Music",English,A promising young drummer enrolls at a cut-thr...,R,2014-10-15,107,Whiplash,2014,8.5,413720,13092000.0
1,"Toshirô Mifune, Takashi Shimura, Keiko Tsushim...",Japan,Akira Kurosawa,"Action, Adventure, Drama",Japanese,A poor village under attack by bandits recruit...,UNRATED,1956-11-19,207,Seven Samurai,1954,8.7,232249,269061.0
2,"Harrison Ford, Karen Allen, Paul Freeman, Rona...",USA,Steven Spielberg,"Action, Adventure","English, German, Hebrew, Spanish, Arabic, Nepali",Archaeologist and adventurer Indiana Jones is ...,PG,1981-06-12,115,Raiders of the Lost Ark,1981,8.5,671034,242374454.0
3,"William Holden, Alec Guinness, Jack Hawkins, S...","UK, USA",David Lean,"Adventure, Drama, War","English, Japanese, Thai",After settling his differences with a Japanese...,PG,1957-12-14,161,The Bridge on the River Kwai,1957,8.2,151604,27200000.0
4,"Robert Downey Jr., Chris Evans, Mark Ruffalo, ...",USA,Joss Whedon,"Action, Sci-Fi, Thriller","English, Russian",Earth's mightiest heroes must come together an...,PG-13,2012-05-04,143,The Avengers,2012,8.1,1010033,623279547.0


## TEXT VECTORIZATION

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
# CountVectorize Genre Column (dummy variables)

cv = CountVectorizer(tokenizer=lambda x: x.split(', '),binary=True, max_features=50)
data = cv.fit_transform(df.Genre).todense()
columns = ['genre_'+c for c in cv.get_feature_names()]
genredf = pd.DataFrame(data, columns=columns)
genredf.head()

,genre_action,genre_adventure,genre_animation,genre_biography,genre_comedy,genre_crime,genre_drama,genre_family,genre_fantasy,genre_film-noir,...,genre_horror,genre_music,genre_musical,genre_mystery,genre_romance,genre_sci-fi,genre_sport,genre_thriller,genre_war,genre_western
0,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [37]:
# Sum of movies by Genre (choose which column to drop)
genredf.sum()

genre_action        39
genre_adventure     62
genre_animation     20
genre_biography     26
genre_comedy        41
genre_crime         55
genre_drama        172
genre_family        14
genre_fantasy       21
genre_film-noir      7
genre_history       13
genre_horror         5
genre_music          1
genre_musical        2
genre_mystery       30
genre_romance       20
genre_sci-fi        23
genre_sport          4
genre_thriller      40
genre_war           20
genre_western        7
dtype: int64

In [38]:
# drop genre_music for classification purposes

genredf.drop('genre_music',axis=1, inplace=True)

In [39]:
df = pd.concat([df, genredf], axis = 1)

In [40]:
# CountVectorize Actor Columns (dummy variables)

cv = CountVectorizer(tokenizer=lambda x: x.split(', '), binary=True, max_features=50)
data = cv.fit_transform(df.Actors).todense()
columns = ['actor: '+c for c in cv.get_feature_names()]
actorsdf = pd.DataFrame(data, columns=columns)
actorsdf.head()

,actor: aamir khan,actor: al pacino,actor: alec guinness,actor: ben kingsley,actor: brad pitt,actor: bruce willis,actor: carrie fisher,actor: carrie-anne moss,actor: charles chaplin,actor: charles vanel,...,actor: robert duvall,actor: russell crowe,actor: sean astin,actor: sigourney weaver,actor: steve buscemi,actor: takashi shimura,actor: tom hanks,actor: tom hardy,actor: toshirô mifune,actor: william holden
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
# Sum of movies by Actor (choose which column to drop)
# Don't need to drop due to already dropped (max 50)

actorsdf.sum()

actor: aamir khan             2
actor: al pacino              4
actor: alec guinness          3
actor: ben kingsley           3
actor: brad pitt              4
actor: bruce willis           3
actor: carrie fisher          4
actor: carrie-anne moss       2
actor: charles chaplin        4
actor: charles vanel          2
actor: charlton heston        2
actor: chris hemsworth        2
actor: christian bale         4
actor: christoph waltz        2
actor: christopher plummer    2
actor: christopher walken     2
actor: clint eastwood         6
actor: gunnar björnstrand     3
actor: harrison ford          7
actor: humphrey bogart        3
actor: jack hawkins           3
actor: jack lemmon            2
actor: jack nicholson         3
actor: james stewart          4
actor: james woods            2
actor: janet leigh            2
actor: jay baruchel           2
actor: joe pesci              4
actor: leonardo dicaprio      7
actor: mark hamill            5
actor: mark ruffalo           3
actor: m

In [42]:
# combine actor and main dataframe

df = pd.concat([df, actorsdf], axis = 1)

In [43]:
# CountVectorize Language Column (dummy variables)

cv = CountVectorizer(tokenizer=lambda x: x.split(', '),binary=True, max_features=50)
data = cv.fit_transform(df.Language).todense()
columns = ['Language: '+c for c in cv.get_feature_names()]
lang_df = pd.DataFrame(data, columns=columns)
lang_df.head()

,Language: american sign language,Language: arabic,Language: awadhi,Language: belarusian,Language: cantonese,Language: czech,Language: danish,Language: english,Language: esperanto,Language: french,...,Language: spanish,Language: swahili,Language: swedish,Language: thai,Language: turkish,Language: urdu,Language: vietnamese,Language: xhosa,Language: yiddish,Language: zulu
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# Sum of movies by Language (choose which column to drop)

lang_df.sum()

Language: american sign language      1
Language: arabic                      8
Language: awadhi                      1
Language: belarusian                  1
Language: cantonese                   5
Language: czech                       1
Language: danish                      2
Language: english                   211
Language: esperanto                   1
Language: french                     42
Language: german                     34
Language: greek                       1
Language: hebrew                      3
Language: hindi                       3
Language: hmong                       1
Language: hungarian                   3
Language: irish                       2
Language: italian                    23
Language: japanese                   19
Language: korean                      2
Language: latin                      10
Language: mandarin                    4
Language: nepali                      1
Language: norwegian                   1
Language: old english                 2


In [45]:
# drop 'Language: american sign language' for classification purposes

lang_df.drop('Language: american sign language',axis=1, inplace=True)

In [46]:
# combine language and main dataframe

df = pd.concat([df, lang_df], axis = 1)

In [47]:
# CountVectorize Country Column (dummy variables)

cv = CountVectorizer(tokenizer=lambda x: x.split(', '),binary=True, max_features=50)
data = cv.fit_transform(df.Country).todense()
columns = ['Country: '+c for c in cv.get_feature_names()]
coun_df = pd.DataFrame(data, columns=columns)
coun_df.head()

,Country: algeria,Country: argentina,Country: australia,Country: austria,Country: belgium,Country: brazil,Country: canada,Country: china,Country: denmark,Country: france,...,Country: south korea,Country: soviet union,Country: spain,Country: sweden,Country: switzerland,Country: taiwan,Country: uk,Country: united arab emirates,Country: usa,Country: west germany
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [48]:
# Sum of movies by country (choose which column to drop)

coun_df.sum()

Country: algeria                   1
Country: argentina                 2
Country: australia                 3
Country: austria                   2
Country: belgium                   1
Country: brazil                    1
Country: canada                    4
Country: china                     1
Country: denmark                   2
Country: france                   20
Country: germany                  14
Country: hong kong                 5
Country: india                     5
Country: iran                      2
Country: ireland                   4
Country: italy                    14
Country: japan                    13
Country: kuwait                    1
Country: lebanon                   1
Country: libya                     1
Country: luxembourg                1
Country: mexico                    2
Country: morocco                   1
Country: new zealand               3
Country: poland                    1
Country: south africa              1
Country: south korea               2
C

In [49]:
# drop 'Country: luxemborg' for classification purposes

coun_df.drop('Country: luxembourg',axis=1, inplace=True)

In [50]:
# combine country and main dataframe

df = pd.concat([df, coun_df], axis = 1)

In [51]:
# CountVectorize Director Column (dummy variables)

cv = CountVectorizer(tokenizer=lambda x: x.split(', '),binary=True, max_features=50)
data = cv.fit_transform(df.Director).todense()
columns = ['Director: '+c for c in cv.get_feature_names()]
direc_df = pd.DataFrame(data, columns=columns)
direc_df.head()

,Director: akira kurosawa,Director: alejandro g. iñárritu,Director: alfred hitchcock,Director: andrew stanton,Director: billy wilder,Director: charles chaplin,Director: christopher nolan,Director: clint eastwood,Director: david fincher,Director: david lean,...,Director: roman polanski,Director: ron howard,Director: sergio leone,Director: sidney lumet,Director: stanley kubrick,Director: steven spielberg,Director: terry gilliam,Director: terry jones,Director: victor fleming,Director: william wyler
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Sum of movies by director (choose which column to drop)
# Don't need to drop due to already dropped (max 50)

direc_df.sum()

Director: akira kurosawa           5
Director: alejandro g. iñárritu    2
Director: alfred hitchcock         7
Director: andrew stanton           2
Director: billy wilder             5
Director: charles chaplin          5
Director: christopher nolan        7
Director: clint eastwood           3
Director: david fincher            3
Director: david lean               2
Director: denis villeneuve         2
Director: ethan coen               3
Director: federico fellini         2
Director: francis ford coppola     3
Director: frank capra              3
Director: frank darabont           2
Director: fritz lang               2
Director: george cukor             2
Director: george roy hill          2
Director: guy ritchie              2
Director: hayao miyazaki           6
Director: henri-georges clouzot    2
Director: ingmar bergman           4
Director: james cameron            3
Director: joel coen                3
Director: john huston              2
Director: lee unkrich              3
D

In [53]:
# combine director and main dataframe

df = pd.concat([df, direc_df], axis = 1)

In [54]:
# drop columns Actors, Genre, Language, Plot, Country, Director

df.drop(['Actors','Genre','Language','Plot','Country','Director'],axis=1, inplace=True)

In [55]:
# Drop duplicates

df.drop_duplicates(inplace=True)

# Exploratory Data Analysis (EDA)

In [57]:
df.describe()

,Runtime,Year,imdbVotes,Gross,genre_action,genre_adventure,genre_animation,genre_biography,genre_comedy,genre_crime,...,Director: roman polanski,Director: ron howard,Director: sergio leone,Director: sidney lumet,Director: stanley kubrick,Director: steven spielberg,Director: terry gilliam,Director: terry jones,Director: victor fleming,Director: william wyler
count,250.000000,250.00000,2.500000e+02,2.500000e+02,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,...,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,129.196000,1984.03600,4.121541e+05,8.063931e+07,0.156000,0.248000,0.080000,0.104000,0.164000,0.220000,...,0.008000,0.008000,0.020000,0.012000,0.028000,0.028000,0.008000,0.008000,0.008000,0.012000
std,32.563524,24.25953,3.389218e+05,1.287048e+08,0.363583,0.432718,0.271837,0.305873,0.371018,0.415077,...,0.089263,0.089263,0.140281,0.109104,0.165304,0.165304,0.089263,0.089263,0.089263,0.109104
min,67.000000,1921.00000,2.627400e+04,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,107.000000,1964.25000,1.218620e+05,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,124.000000,1991.00000,3.302050e+05,2.118502e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,143.000000,2003.75000,6.027400e+05,1.171431e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,320.000000,2016.00000,1.711064e+06,9.366274e+08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [84]:
pd.pivot_table(df,index='Title', values=['Gross'], aggfunc='max')

,Gross
Title,
12 Angry Men,0.0
12 Years a Slave,56667870.0
2001: A Space Odyssey,56715371.0
3 Idiots,6523137.0
8½,50690.0
A Beautiful Mind,170708996.0
A Clockwork Orange,0.0
A Fistful of Dollars,3500000.0
A Separation,7098492.0


# Decision Trees

In [69]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.grid_search import GridSearchCV

from patsy import dmatrices
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


//anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
X = 
y = df['Rating']

In [ ]:
df.plot('ggplot')

In [ ]:
8.5 